<a href="https://colab.research.google.com/github/saieswaramurali/ATM_machine_with_face_verification/blob/main/ATM_WITH_FACE_VERIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
customer_database = []
customer_known_img = []
customer_Unknown_img = [] # assuming this is the img calculated from the user while withdrawing.

# think a way to avoid users with same name ; ------ DONE
# should implement OTP for verification ------ DONE
# and add face recognition for two step verification ----- DONE


In [ ]:
!pip install twilio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.9 MB/s eta 0:00:00


In [ ]:
!pip install face_recognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=0f473df02119a11a452cd0df2ee72ef17ace40638a35be4fa2b346b364932466
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
!mkdir known
!mkdir Unknown

In [ ]:
import face_recognition
import cv2
import os
from google.colab.patches import cv2_imshow

def read_img(path) :
  img = cv2.imread(path)
  (h, v) = img.shape[:2] # getting height and width
  width = 500
  ratio =  width / float(v)
  height = int(h * ratio)
  return cv2.resize(img, [width, height])

known_encodings = []
known_names = []
known_dir = "known"

for file in os.listdir(known_dir) :
  img = read_img(known_dir + '/' + file)
  img_enc = face_recognition.face_encodings(img)[0]
  known_encodings.append(img_enc)
  known_names.append(file.split('.')[0])

unknown_dir = 'Unknown'

for file in os.listdir(unknown_dir) :
  print('Processing', file)
  img = read_img(unknown_dir + '/' + file)
  img_enc = face_recognition.face_encodings(img)[0]

  results = face_recognition.compare_faces(known_encodings, img_enc)

  for i in range(len(results)) :
    if results[i] :
      print(known_names[i])

  print(results)




Processing jeff_bezoz.jpg
JEFF
[False, True]
Processing elon_musk.jpg
ELON
[True, False]


In [ ]:
import random
from twilio.rest import Client

#for sending opt

def send_otp(contact) :
    otp = random.randint(1000,9999)

    account_sid = "AC70f4dc175df497d414dd6f32_____" # enter your account sid from twilio account here to send otp

    auth_token = "2e0e5e164d5b500bc5eb2c4889____"    # enter your account's authentication token from twilio account here to send otp

    client = Client(account_sid, auth_token)

    msg = client.messages.create(
        body = f"Welcome to our bank, Your OTP is {otp}" ,
        from_ = "+15733672585",
        to = f"+{contact}"
    )
    return otp



In [ ]:
#think of a way to avoid users with same username ;
for i in customer_database :
  print(i.username)

saie
vimal
sai
elon
jeff
elon
jeff


In [ ]:
from prompt_toolkit.application.application import E

import math


#class for customers
class customer :

  def __init__(self, username, pin, contact, balance = 0) :
    assert int(math.log10(pin) + 1) == 4 , "pin number is not valid"
    #assert len(username) <= 3, "username cannot be empty"
    assert balance >= 0 , "balance cannot be negative"
    assert int(math.log10(contact)) + 1 == 12 , "Enter contact number with the country code ex: +91______1234"
    self.username = username
    self.pin = pin
    self.balance = balance
    self.contact = contact

#function to display menu

def start_page() :
  print("""
    WELCOME TO OUR BANK
    ENTER 1 TO LOGIN TO AN EXISTING ACCOUNT
    ENTER 2 TO CREATE AN ACCOUNT
  """)

def Menu() :

  # print("Welcome to our bank")
    print("""
        WELCOME TO OUR BANK
        ****MENU****
        Enter 1 to check balance
        Enter 2 to withdraw the cash
        Enter 3 to deposit the cash
        Enter 4 to cancel
    """)
    choice = int(input())
    while(type(choice) != type(0)) :
      choice = int(input())
    name = str(input("Enter the username : "))
    pin = int(input("Enter the pin : "))


    database_index = None
    # to check whether the user is already exist in the database ;
    flag = 0
    for i in range(len(customer_database)) :
      if customer_database[i].username == name :
        flag = 1
        database_index = i
        break
    if flag == 0 :
      print("!!! KINDLY CHECK THE USERNAME ONCE AGAIN!!!")

    if flag == 1 : #user exists  ;



          if customer_database[database_index].pin == pin :
          # implement multiple verifications like OTP and face verification by adding if statements here ;

          #for OTP verification ;
                otp_sent = send_otp(customer_database[database_index].contact)
                otp_entered = int(input("Enter the otp recieved in your contact number. "))

                if otp_entered == otp_sent :

                    if choice == 4 : # to cancel
                      pass
                    elif choice == 3 : # to deposit the cash
                      deposit = int(input("Enter the amount to deposit : "))
                      while deposit <= 0 :
                        print("Enter appropriate amount to deposit")
                        deposit = int(input("Enter the amount to deposit : "))

                      customer_database[database_index].balance += deposit
                      print(f"HENCE the updated balance of account is : {customer_database[database_index].balance}")

                    elif choice == 2 : # to withdraw the cash
                      withdraw = int(input("Enter the amount to withdraw : "))

                      while withdraw <= 0 or customer_database[database_index].balance < withdraw : #invalid amount to withdraw
                        print("invalid balance or invalid amount entered")
                        withdraw = int(input("Enter the amount to withdraw : "))

                      if (withdraw >= 15000) :
                        img = read_img(unknown_dir + '/' + 'elon_musk.jpg' )
                        img_enc = face_recognition.face_encodings(img)

                        result = face_recognition.compare_faces(known_encodings[1], img_enc)

                        if results :
                          print("Client face is recognised")
                          customer_database[database_index].balance -= withdraw
                          print(f"HENCE the updated balance of account is {customer_database[database_index].balance}")
                        else :
                          print("!!!The Client face is not recognisied!!!")


                      else :
                        customer_database[database_index].balance -= withdraw
                        print(f"HENCE the updated balance of account is {customer_database[database_index].balance}")

                    elif choice == 1 : # to check the balance
                      print(f"Your account balance is {customer_database[database_index].balance}")

                else :
                    print("!!! Entered OTP is wrong !!!")



          else :
            print("!!! kindly check your pin number !!!")






#function to initiate an account for new users
def create_account(username, pin, contact, balance) :
    assert int(math.log10(pin) + 1) == 4 , "pin number is not valid"
    #assert len(username) == 0, "username cannot be empty"
    assert balance >= 0 , "balance cannot be negative"
    #assert username in customer_database.username() , "username is taken already"

    customer_database.append(customer(username, pin, contact, balance))


def Main() :
  start_page()
  choice = int(input())

  if choice == 1 :
    #display the information of the existing user :
    Menu()
  elif choice == 2:
    #want to create a new account for user
    name = str(input("Enter the username : "))
    pin = int(input("Enter the pin : "))
    balance = int(input("Enter the balance : "))
    contact = int(input("Enter your contact number : "))
    create_account(name, pin, contact,  balance)



#sai = customer("sai", 1222, 9000)
#should try to implement a off button :
#while True :

Main()

# to clear the database if we need to ;
def clear_userDatabase() :
  customer_database.clear()


    WELCOME TO OUR BANK
    ENTER 1 TO LOGIN TO AN EXISTING ACCOUNT
    ENTER 2 TO CREATE AN ACCOUNT
  
1

        WELCOME TO OUR BANK
        ****MENU****
        Enter 1 to check balance
        Enter 2 to withdraw the cash
        Enter 3 to deposit the cash
        Enter 4 to cancel
    
2
Enter the username : saie
Enter the pin : 1234
Enter the otp recieved in your contact number. 9275
Enter the amount to withdraw : 25000
Client face is recognised
HENCE the updated balance of account is 87000


In [ ]:
# inititate the customer_database with some client objects after running the main method once
print(len(customer_database))
print(customer_database[0].contact)
#customer_database.clear()

create_account("elon", 1234, 919444488518, 0)
create_account("jeff", 1234, 918778016026, 0)



5
919444488518


In [ ]:
print(len(customer_database))

print(customer_database[0].contact)


2
919444488518
